In [ ]:
import pandas as pd 
import numpy as np 
from selenium import webdriver 
from tqdm import tqdm 

from bball_reference import BBallScrape 



In [ ]:
def ReadFiles(year_range): 
    
    try: 

        player_salary = pd.read_csv('csv_files/player_salary.csv') 
        print('player salary worked') 

        player_stats = pd.read_csv('csv_files/player_stats.csv') 
        print('player_stats worked') 

        standings = pd.read_csv('csv_files/standings.csv') 
        print('standings worked') 

        team_stats = pd.read_csv('csv_files/team_stats') 
        print('team_stats worked') 

        opponent_stats = pd.read_csv('csv_files/opponent_stats.csv') 
        print('opponent_stats worked') 
    

    except: 

        all_stats = BBallScrape(year_range[0]).ReturnAllData()

        player_salary = all_stats['player_salary'] 
        player_stats = all_stats['player_stats'] 
        standings = all_stats['standings'] 
        team_stats = all_stats['team_stats'] 
        opponent_stats = all_stats['opponent_stats'] 

        for year in tqdm(year_range[1:]): 
            

            player_salary = player_salary.append(all_stats['player_salary'] ) 
            player_stats = player_stats.append(all_stats['player_stats'] ) 
            standings = standings.append(all_stats['standings'] ) 
            team_stats = team_stats.append(all_stats['team_stats'] ) 
            opponent_stats = opponent_stats.append(all_stats['opponent_stats'] ) 

        player_salary.to_csv('csv_files/player_salary') 
        player_stats.to_csv('csv_files/player_stats') 
        standings.to_csv('csv_files/standings') 
        team_stats.to_csv('csv_files/team_stats') 
        opponent_stats.to_csv('csv_files/opponent_stats') 

    return {'player_salary': player_salary, 
            'player_stats': player_stats, 
            'standings': standings, 
            'team_stats': team_stats, 
            'opponent_stats': opponent_stats} 



year_range = np.arange(2000, 2020, 1)
dfs = ReadFiles(year_range) 


In [ ]:
pd.read_csv('csv_files/opponent_stats.csv', encoding = 'latin-1')

In [ ]:
pd.read_csv('csv_files/opponent_stats.xlsx') 